# Dynamic Prompts and Middleware in LangChain 1.0

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EnkrateiaLucca/oreilly_live_training_getting_started_with_langchain/blob/main/notebooks/6.0-dynamic-prompts-middleware.ipynb)

In this notebook, we'll explore **middleware** and **dynamic prompts** in LangChain 1.0. These powerful features allow you to:

- Intercept and modify agent behavior at runtime
- Implement role-based access control
- Add logging, rate limiting, and custom business logic
- Change system prompts dynamically based on context

## Learning Objectives

By the end of this notebook, you'll be able to:

1. Understand the middleware architecture in LangChain 1.0
2. Use `@dynamic_prompt` to modify prompts based on runtime context
3. Implement role-based access control for agents
4. Create custom middleware for logging and monitoring
5. Compose multiple middleware together

## Setup and Installation

In [ ]:
# LangChain 1.0+ Setup
# %pip install -qU langchain>=1.0.0
# %pip install -qU langchain-core>=1.0.0
# %pip install -qU langchain-openai
# %pip install -qU langgraph>=1.0.0

## API Key Setup

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

OpenAI API quickstart: https://platform.openai.com/docs/quickstart

## 1. Introduction: What is Middleware?

### Understanding Middleware

In LangChain 1.0, **middleware** is code that runs at specific points during agent execution. Think of it as a layer that sits between your input and the agent's processing.

```
User Input
    ↓
Middleware Layer (intercepts)
    ↓
Agent Processing
    ↓
Middleware Layer (intercepts)
    ↓
Output
```

### Common Use Cases

1. **Logging & Monitoring**: Track agent decisions and tool calls
2. **Rate Limiting**: Control API usage and costs
3. **Dynamic Prompts**: Change system prompts based on user context
4. **Access Control**: Restrict tool access based on user roles
5. **Content Filtering**: Screen inputs/outputs for sensitive information
6. **Error Handling**: Gracefully handle failures with fallbacks

### How Middleware Intercepts Execution

Middleware can intercept at two key points:

1. **Before model calls** - Modify prompts, check permissions
2. **After tool execution** - Log results, validate outputs

Let's see this in action!

## 2. RuntimeContext Review

Before diving into middleware, let's quickly review **RuntimeContext** - the dependency injection pattern we use to pass context to agents.

### What is RuntimeContext?

RuntimeContext is a dataclass that holds runtime information (like database connections, user info, configuration) that your agent and tools can access.

In [ ]:
from dataclasses import dataclass
from langchain_core.tools import tool
from langchain.agents import get_runtime

# Define a simple RuntimeContext
@dataclass
class UserContext:
    """Context about the current user."""
    user_id: str
    username: str
    role: str  # "admin" or "user"

# Example tool that uses context
@tool
def get_user_info() -> str:
    """Get information about the current user."""
    runtime = get_runtime(UserContext)
    ctx = runtime.context
    return f"User: {ctx.username} (ID: {ctx.user_id}, Role: {ctx.role})"

print("Tool defined successfully!")

### How get_runtime() Works

Inside any tool, you can call `get_runtime(YourContextClass)` to access the context that was passed when invoking the agent:

```python
# When invoking the agent:
agent.invoke(
    {"messages": "Hello"},
    context=UserContext(user_id="123", username="alice", role="admin")
)
```

The context flows through the agent to your tools automatically!

## 3. Dynamic Prompts with @dynamic_prompt

Now let's explore the most powerful middleware feature: **dynamic prompts**.

### The Problem

Imagine you have a SQL agent that should behave differently for different users:
- **Admins** should see all tables and have full access
- **Regular users** should only see certain tables

With static prompts, you'd need separate agents for each role. With dynamic prompts, you can use **one agent** that adapts its behavior!

### The @dynamic_prompt Decorator

The `@dynamic_prompt` decorator lets you write a function that returns different prompts based on the runtime context.

In [ ]:
# Note: As of LangChain 1.0, middleware APIs may be experimental
# Check latest docs at: https://docs.langchain.com/oss/python/langchain/agents

# For this demo, we'll show the conceptual pattern
# The actual implementation may vary based on the final LangChain 1.0 API

# Conceptual pattern for dynamic prompts:
from typing import Any

# This is a simplified representation of the ModelRequest object
class ModelRequest:
    """Contains information about the current model call."""
    def __init__(self, runtime, messages):
        self.runtime = runtime  # Contains the RuntimeContext
        self.messages = messages  # The current conversation messages

print("ModelRequest class defined for demonstration")

### Understanding ModelRequest

When you use `@dynamic_prompt`, your function receives a `ModelRequest` object with:

- `request.runtime.context` - Your RuntimeContext (e.g., user info)
- `request.messages` - The current conversation messages
- Other metadata about the model call

Your function should return the system prompt as a string.

In [ ]:
# Example: Dynamic prompt based on user role

ADMIN_PROMPT = """You are a SQL database assistant with FULL ACCESS to all tables.

Available tables:
- customers (id, name, email, created_at)
- orders (id, customer_id, total, status, created_at)
- products (id, name, price, inventory)
- internal_metrics (sensitive financial data)

You can query any table and provide detailed insights.
"""

USER_PROMPT = """You are a SQL database assistant with LIMITED ACCESS.

Available tables (read-only):
- customers (id, name, email) - You can only see customer ID and name
- orders (id, customer_id, total, status)
- products (id, name, price)

IMPORTANT: You CANNOT access:
- internal_metrics table (confidential)
- customer email addresses
- detailed financial data

If asked for restricted data, politely explain you don't have access.
"""

# The dynamic prompt function (conceptual pattern)
def dynamic_prompt_function(request: ModelRequest) -> str:
    """Return different prompts based on user role."""
    # Access the user context from the request
    user_context = request.runtime.context
    
    # Choose prompt based on role
    if user_context.role == "admin":
        return ADMIN_PROMPT
    else:
        return USER_PROMPT

print("Dynamic prompt function created!")

### Key Insight

Notice how the prompt changes **dynamically** based on `user_context.role`. The agent doesn't need to know about roles - the middleware handles it!

This means:
- ✅ One agent codebase for all users
- ✅ Security enforced at the prompt level
- ✅ Easy to add new roles (just update the function)

## 4. Role-Based Access Control Example

Let's build a complete example with role-based access control. We'll create:

1. A simple SQL-like tool
2. An agent that uses dynamic prompts
3. Test it with admin and regular user contexts

In [ ]:
# Mock database for demonstration
MOCK_DB = {
    "customers": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
    ],
    "orders": [
        {"id": 101, "customer_id": 1, "total": 99.99, "status": "completed"},
        {"id": 102, "customer_id": 2, "total": 149.99, "status": "pending"},
    ],
    "internal_metrics": [
        {"metric": "revenue", "value": 1000000, "sensitive": True},
    ],
}

@tool
def query_table(table_name: str) -> str:
    """Query a database table by name. Returns JSON data."""
    runtime = get_runtime(UserContext)
    ctx = runtime.context
    
    # Check if table exists
    if table_name not in MOCK_DB:
        return f"Error: Table '{table_name}' not found."
    
    # Access control: non-admins can't access internal_metrics
    if table_name == "internal_metrics" and ctx.role != "admin":
        return "Error: Access denied. You don't have permission to view this table."
    
    # Filter sensitive data for non-admins
    data = MOCK_DB[table_name]
    if ctx.role != "admin" and table_name == "customers":
        # Remove email for non-admins
        data = [{k: v for k, v in row.items() if k != "email"} for row in data]
    
    import json
    return json.dumps(data, indent=2)

print("Query tool created!")

### Creating the Agent with Dynamic Prompts

Now let's create an agent. In LangChain 1.0, you would pass middleware to `create_agent()`:

```python
# Conceptual pattern (API may vary in final LangChain 1.0)
from langchain.agents import create_agent
from langchain.agents.middleware.types import dynamic_prompt

@dynamic_prompt
def role_based_prompt(request: ModelRequest) -> str:
    if request.runtime.context.role == "admin":
        return ADMIN_PROMPT
    return USER_PROMPT

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[query_table],
    context_schema=UserContext,
    middleware=[role_based_prompt],  # Pass the middleware here
)
```

Since the final middleware API is still evolving in LangChain 1.0, let's demonstrate the **concept** using a simpler approach:

In [ ]:
from langchain.agents import create_agent

# For now, we'll use the system_prompt parameter directly
# In production with full middleware support, you'd use the @dynamic_prompt pattern

def create_role_based_agent(user_role: str):
    """Create an agent with a prompt based on user role."""
    
    # Choose prompt based on role
    system_prompt = ADMIN_PROMPT if user_role == "admin" else USER_PROMPT
    
    agent = create_agent(
        model="openai:gpt-4o-mini",
        tools=[query_table, get_user_info],
        system_prompt=system_prompt,
        context_schema=UserContext,
    )
    
    return agent

print("Agent factory created!")

### Testing with Admin Context

In [ ]:
# Create admin agent
admin_agent = create_role_based_agent("admin")

# Create admin context
admin_context = UserContext(
    user_id="001",
    username="admin_alice",
    role="admin"
)

# Test: Admin requesting sensitive data
result = admin_agent.invoke(
    {"messages": "Show me the internal_metrics table"},
    context=admin_context
)

print("\n=== ADMIN REQUEST ===")
print(result["messages"][-1].content)

### Testing with Regular User Context

In [ ]:
# Create regular user agent
user_agent = create_role_based_agent("user")

# Create user context
user_context = UserContext(
    user_id="002",
    username="bob_user",
    role="user"
)

# Test: Regular user requesting sensitive data
result = user_agent.invoke(
    {"messages": "Show me the internal_metrics table"},
    context=user_context
)

print("\n=== USER REQUEST (Should be denied) ===")
print(result["messages"][-1].content)

### Testing Data Filtering

In [ ]:
# Admin can see emails
result = admin_agent.invoke(
    {"messages": "Show me the customers table"},
    context=admin_context
)

print("\n=== ADMIN: Customers (with emails) ===")
print(result["messages"][-1].content)

In [ ]:
# Regular user cannot see emails
result = user_agent.invoke(
    {"messages": "Show me the customers table"},
    context=user_context
)

print("\n=== USER: Customers (no emails) ===")
print(result["messages"][-1].content)

## 5. Custom Middleware: Logging Example

Beyond dynamic prompts, you can create custom middleware for other purposes. Let's build a **logging middleware** that tracks all agent actions.

### Conceptual Pattern

```python
# Conceptual middleware pattern
from langchain.agents.middleware.types import Middleware

class LoggingMiddleware(Middleware):
    def before_model_call(self, request: ModelRequest):
        """Called before each LLM call."""
        print(f"[LOG] Model call with {len(request.messages)} messages")
    
    def after_tool_call(self, tool_name: str, result: Any):
        """Called after each tool execution."""
        print(f"[LOG] Tool '{tool_name}' returned: {result[:100]}...")
```

In [ ]:
# Simple logging implementation using callbacks
# (Full middleware API may differ in LangChain 1.0)

from datetime import datetime

class SimpleLogger:
    """Simple logger for demonstration."""
    
    def __init__(self):
        self.logs = []
    
    def log_action(self, action: str, details: str):
        """Log an action with timestamp."""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"[{timestamp}] {action}: {details}"
        self.logs.append(log_entry)
        print(log_entry)
    
    def get_logs(self):
        """Return all logs."""
        return "\n".join(self.logs)

# Create a global logger instance
logger = SimpleLogger()

print("Logger created!")

In [ ]:
# Create a tool that uses the logger
@tool
def logged_query_table(table_name: str) -> str:
    """Query a database table with logging."""
    runtime = get_runtime(UserContext)
    ctx = runtime.context
    
    # Log the request
    logger.log_action(
        "TOOL_CALL",
        f"User {ctx.username} (role={ctx.role}) querying table '{table_name}'"
    )
    
    # Check permissions
    if table_name not in MOCK_DB:
        logger.log_action("ERROR", f"Table '{table_name}' not found")
        return f"Error: Table '{table_name}' not found."
    
    if table_name == "internal_metrics" and ctx.role != "admin":
        logger.log_action("ACCESS_DENIED", f"User {ctx.username} denied access to {table_name}")
        return "Error: Access denied. You don't have permission to view this table."
    
    # Get data
    data = MOCK_DB[table_name]
    if ctx.role != "admin" and table_name == "customers":
        data = [{k: v for k, v in row.items() if k != "email"} for row in data]
    
    logger.log_action("SUCCESS", f"Returned {len(data)} rows from {table_name}")
    
    import json
    return json.dumps(data, indent=2)

print("Logged query tool created!")

In [ ]:
# Create agent with logging tool
logged_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[logged_query_table],
    system_prompt=USER_PROMPT,
    context_schema=UserContext,
)

# Test it
result = logged_agent.invoke(
    {"messages": "Show me customers and then try to access internal_metrics"},
    context=user_context
)

print("\n=== AGENT RESPONSE ===")
print(result["messages"][-1].content)

In [ ]:
# View all logs
print("\n=== COMPLETE LOG ===")
print(logger.get_logs())

## 6. Rate Limiting Concept

Another common use case for middleware is **rate limiting** - controlling how often users can call expensive operations.

### Conceptual Implementation

In [ ]:
from collections import defaultdict
import time

class RateLimiter:
    """Simple rate limiter for demonstration."""
    
    def __init__(self, max_calls: int = 5, window_seconds: int = 60):
        self.max_calls = max_calls
        self.window_seconds = window_seconds
        self.calls = defaultdict(list)  # user_id -> [timestamps]
    
    def is_allowed(self, user_id: str) -> tuple[bool, str]:
        """Check if user is within rate limits."""
        now = time.time()
        
        # Remove old calls outside the window
        self.calls[user_id] = [
            t for t in self.calls[user_id] 
            if now - t < self.window_seconds
        ]
        
        # Check if under limit
        if len(self.calls[user_id]) >= self.max_calls:
            return False, f"Rate limit exceeded. Max {self.max_calls} calls per {self.window_seconds}s."
        
        # Record this call
        self.calls[user_id].append(now)
        remaining = self.max_calls - len(self.calls[user_id])
        return True, f"OK. {remaining} calls remaining."

# Create rate limiter
rate_limiter = RateLimiter(max_calls=3, window_seconds=10)

print("Rate limiter created (3 calls per 10 seconds)")

In [ ]:
# Tool with rate limiting
@tool
def rate_limited_query(table_name: str) -> str:
    """Query a table with rate limiting."""
    runtime = get_runtime(UserContext)
    ctx = runtime.context
    
    # Check rate limit
    allowed, message = rate_limiter.is_allowed(ctx.user_id)
    logger.log_action("RATE_LIMIT_CHECK", f"User {ctx.user_id}: {message}")
    
    if not allowed:
        return f"Error: {message}"
    
    # Proceed with query
    if table_name not in MOCK_DB:
        return f"Error: Table '{table_name}' not found."
    
    import json
    return json.dumps(MOCK_DB[table_name], indent=2)

print("Rate-limited tool created!")

In [ ]:
# Test rate limiting
rate_limited_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[rate_limited_query],
    system_prompt="You are a helpful assistant. Query tables as requested.",
    context_schema=UserContext,
)

# Make several rapid requests
for i in range(5):
    print(f"\n=== Request {i+1} ===")
    result = rate_limited_agent.invoke(
        {"messages": f"Show me the customers table (request {i+1})"},
        context=user_context
    )
    print(result["messages"][-1].content)
    time.sleep(0.5)  # Small delay between requests

## 7. Middleware Composition

In production systems, you often want to combine multiple middleware together. For example:

1. Rate limiting (first)
2. Logging (second)
3. Dynamic prompts (third)

### Order of Execution

Middleware executes in the order you pass it to `create_agent()`:

```python
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[...],
    middleware=[
        rate_limiting_middleware,  # Runs first
        logging_middleware,         # Runs second
        dynamic_prompt_middleware,  # Runs third
    ],
)
```

### Why Order Matters

```
Rate Limiting → Logging → Dynamic Prompts → Model Call
```

- **Rate limiting first**: Reject requests before doing any work
- **Logging second**: Log all requests that pass rate limiting
- **Dynamic prompts third**: Modify prompt based on user context

If you reversed the order, you'd waste resources generating prompts for rate-limited requests!

## 8. Practical Example: Complete Role-Based System

Let's put everything together into a production-ready example with:

1. Role-based dynamic prompts
2. Access control in tools
3. Logging
4. Rate limiting

### The Complete System

In [ ]:
# Enhanced context with more fields
@dataclass
class EnhancedUserContext:
    """Enhanced user context with more information."""
    user_id: str
    username: str
    role: str  # "admin", "analyst", "user"
    department: str  # "engineering", "sales", "finance"
    permissions: list[str]  # Explicit permissions

# Define prompts for different roles
PROMPTS = {
    "admin": """You are a SQL assistant with FULL ACCESS to all data.
You can query any table and provide any information requested.
Available tables: customers, orders, products, internal_metrics, salaries""",
    
    "analyst": """You are a SQL assistant for data analysts.
You can access: customers, orders, products
You CANNOT access: internal_metrics, salaries
Focus on business insights and reporting.""",
    
    "user": """You are a SQL assistant with limited access.
You can access: orders, products (read-only)
You CANNOT access: customers, internal_metrics, salaries
If asked for restricted data, explain you don't have access.""",
}

print("Enhanced context and prompts defined!")

In [ ]:
# Comprehensive tool with all checks
@tool
def secure_query(table_name: str) -> str:
    """Securely query a database table with all safety checks."""
    runtime = get_runtime(EnhancedUserContext)
    ctx = runtime.context
    
    # 1. Rate limiting
    allowed, rate_message = rate_limiter.is_allowed(ctx.user_id)
    if not allowed:
        logger.log_action("RATE_LIMIT_EXCEEDED", f"User {ctx.username}: {rate_message}")
        return f"Error: {rate_message}"
    
    # 2. Logging
    logger.log_action(
        "QUERY_REQUEST",
        f"User: {ctx.username}, Role: {ctx.role}, Table: {table_name}"
    )
    
    # 3. Permission check
    table_permission = f"query_{table_name}"
    if table_permission not in ctx.permissions and ctx.role != "admin":
        logger.log_action("ACCESS_DENIED", f"User {ctx.username} lacks permission: {table_permission}")
        return f"Error: You don't have permission to access '{table_name}'. Contact your administrator."
    
    # 4. Table existence check
    if table_name not in MOCK_DB:
        logger.log_action("TABLE_NOT_FOUND", f"Table '{table_name}' does not exist")
        return f"Error: Table '{table_name}' not found in the database."
    
    # 5. Execute query
    data = MOCK_DB[table_name]
    
    # 6. Data filtering based on role
    if table_name == "customers" and ctx.role == "analyst":
        # Analysts can't see emails
        data = [{k: v for k, v in row.items() if k != "email"} for row in data]
    
    logger.log_action("QUERY_SUCCESS", f"Returned {len(data)} rows from {table_name}")
    
    import json
    return json.dumps(data, indent=2)

print("Secure query tool created!")

In [ ]:
# Create agents for different roles
def create_secure_agent(role: str):
    """Create agent with role-appropriate prompt."""
    return create_agent(
        model="openai:gpt-4o-mini",
        tools=[secure_query],
        system_prompt=PROMPTS[role],
        context_schema=EnhancedUserContext,
    )

# Define user contexts
contexts = {
    "admin": EnhancedUserContext(
        user_id="001",
        username="admin_alice",
        role="admin",
        department="engineering",
        permissions=["query_customers", "query_orders", "query_products", "query_internal_metrics", "query_salaries"],
    ),
    "analyst": EnhancedUserContext(
        user_id="002",
        username="analyst_bob",
        role="analyst",
        department="finance",
        permissions=["query_customers", "query_orders", "query_products"],
    ),
    "user": EnhancedUserContext(
        user_id="003",
        username="user_charlie",
        role="user",
        department="sales",
        permissions=["query_orders", "query_products"],
    ),
}

print("Agents and contexts ready!")

### Test the Complete System

In [ ]:
# Test 1: Admin can access everything
admin_agent = create_secure_agent("admin")
result = admin_agent.invoke(
    {"messages": "Show me the internal_metrics table"},
    context=contexts["admin"]
)
print("\n=== TEST 1: Admin accessing internal_metrics ===")
print(result["messages"][-1].content)

In [ ]:
# Test 2: Analyst cannot access internal_metrics
analyst_agent = create_secure_agent("analyst")
result = analyst_agent.invoke(
    {"messages": "Show me the internal_metrics table"},
    context=contexts["analyst"]
)
print("\n=== TEST 2: Analyst trying to access internal_metrics ===")
print(result["messages"][-1].content)

In [ ]:
# Test 3: Analyst can access customers (without emails)
result = analyst_agent.invoke(
    {"messages": "Show me the customers table"},
    context=contexts["analyst"]
)
print("\n=== TEST 3: Analyst accessing customers (emails filtered) ===")
print(result["messages"][-1].content)

In [ ]:
# Test 4: Regular user cannot access customers
user_agent = create_secure_agent("user")
result = user_agent.invoke(
    {"messages": "Show me the customers table"},
    context=contexts["user"]
)
print("\n=== TEST 4: User trying to access customers ===")
print(result["messages"][-1].content)

In [ ]:
# View complete audit log
print("\n=== COMPLETE AUDIT LOG ===")
print(logger.get_logs())

## 9. Key Takeaways

### What We Learned

1. **Middleware Architecture**
   - Middleware intercepts agent execution at key points
   - Used for logging, rate limiting, dynamic prompts, access control
   - Runs before model calls and after tool execution

2. **Dynamic Prompts**
   - Use `@dynamic_prompt` to change prompts based on runtime context
   - Access context via `request.runtime.context`
   - Return different prompts for different users/scenarios

3. **Role-Based Access Control**
   - Combine dynamic prompts + permission checks in tools
   - Filter data based on user roles
   - One agent codebase serves all user types

4. **Custom Middleware**
   - Logging: Track all agent actions for auditing
   - Rate limiting: Prevent abuse and control costs
   - Content filtering: Screen sensitive information

5. **Middleware Composition**
   - Order matters: rate limiting → logging → dynamic prompts
   - Middleware runs in sequence
   - Early middleware can prevent later middleware from running

### Production Patterns

```python
# Complete production-ready agent setup
agent = create_agent(
    model="openai:gpt-4o",
    tools=[secure_tool1, secure_tool2],
    context_schema=UserContext,
    middleware=[
        rate_limiting_middleware,
        logging_middleware,
        dynamic_prompt_middleware,
    ],
    checkpointer=PostgresSaver(),  # Persistent memory
)
```

### Important Notes

⚠️ **Note**: Some middleware APIs shown here are conceptual patterns. The exact API in LangChain 1.0 may vary. Always check the latest documentation:
- https://docs.langchain.com/oss/python/langchain/agents
- https://python.langchain.com/docs/how_to/#agents

The core concepts (dynamic prompts, middleware composition, access control) remain the same, but implementation details may differ.

## 10. Exercises

Try these exercises to deepen your understanding:

### Exercise 1: Add a New Role

Add a "manager" role that can:
- Access customers, orders, products
- Access internal_metrics but NOT salaries
- See customer emails

### Exercise 2: Content Filter Middleware

Create middleware that:
- Scans tool outputs for sensitive keywords ("password", "ssn", "credit_card")
- Redacts or blocks outputs containing these terms
- Logs any redactions

### Exercise 3: Time-Based Access

Modify the dynamic prompt to:
- Grant extra permissions during business hours (9 AM - 5 PM)
- Restrict access outside business hours
- Use Python's `datetime` module

### Exercise 4: Department-Based Data Filtering

Create a tool that:
- Filters orders by department
- Sales department only sees sales data
- Engineering department only sees engineering data
- Admins see everything

## 11. Next Steps

Now that you understand middleware and dynamic prompts, you can:

1. **Explore Human-in-the-Loop**: Check out `HumanInTheLoopMiddleware` for requiring approval before tool execution

2. **Build Production Systems**: Combine middleware with:
   - PostgreSQL checkpointers for persistent memory
   - LangSmith for observability
   - Structured output for guaranteed response formats

3. **Advanced Middleware**: Create custom middleware for:
   - A/B testing different prompts
   - Cost tracking and budgets
   - Multi-tenant isolation

4. **Read the Docs**: 
   - [LangChain Agents](https://docs.langchain.com/oss/python/langchain/agents)
   - [LangGraph Checkpointers](https://langchain-ai.github.io/langgraph/)

Happy building! 🚀